#### To Do
- download each csv file to ../tmp folder
- add ../tmp to strblob

In [75]:
import pyspark
from pyspark.sql import SparkSession
from prefect import flow, task
from google.cloud import storage
import pandas as pd

In [76]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [77]:
catch_dtype = {'common': str, 'strat_id': str, 'psu_id': int, 
                'id_code': int, 'MODE_FX': int, 'AREA_X': int, 
                'ST': int, 'SUB_REG': int, 'WAVE': int, 
                'year': int, 'month': int, 'kod': str, 
                'SP_CODE': float, 'CLAIM': float, 'RELEASE': float, 
                'HARVEST': float, 'CLAIM_UNADJ': int, 'HARVEST_UNADJ': int, 
                'RELEASE_UNADJ': int, 'tot_len_a': float, 'wgt_a': float, 
                'tot_len_b1': float, 'wgt_b1': float, 'tot_cat': float, 
                'wgt_ab1': float, 'tot_len': float, 'Landing': float, 
                'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
                'fl_reg': int, 'wp_catch_precal': float, 'wp_int': float, 
                'wp_catch': float, 'date_published': str}

trip_dtype = {'prim2_common': str, 'prim1_common': str, 'strat_id': str, 
              'psu_id': int, 'ADD_HRS': float, 'AREA': int, 
              'AREA_X': int, 'BOAT_HRS': float, 'CATCH': int, 
              'CNTRBTRS': int, 'CNTY': int, 'CNTY_RES': int, 
              'COASTAL': str, 'FFDAYS2': int, 'FFDAYS12': int, 
              'FIRST': float, 'HRSF': float, 'ID_CODE': int, 
              'INTSITE': int, 'MODE_F': int, 'MODE_FX': int, 
              'NUM_TYP2': int, 'NUM_TYP3': int, 'NUM_TYP4': int, 
              'NUM_TYP6': float, 'ON_LIST': float, 'PARTY': float, 
              'REG_RES': float, 'ST': int, 'ST_RES': int, 
              'SUB_REG': int, 'TELEFON': int, 'WAVE': int, 
              'YEAR': int, 'ASG_CODE': int, 'month': int, 
              'kod': str, 'MODE_ASG': float, 'new_list': float, 
              'PRT_CODE': int, 'CELLTYPE': float, 'fshinsp_a': float, 
              'num_fish_a': float, 'fl_reg': int, 'ADD_PH': int, 
              'AREA_NC': float, 'COUNTY': str, 'DATE1': str, 
              'DIST': int, 'F_BY_P': int, 'GEAR': int, 
              'MODE2001': float, 'MUNI_RES': float, 'MUNI_TRP': float, 
              'PRIM1': float, 'PRIM2': float, 'PVT_RES': int, 
              'RIG': int, 'SEP_FISH': int, 'TIME': int, 
              'ZIP': float, 'AGE': float, 'wp_int': float, 
              'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
              'LEADER': int, 'date_published': str}

size_dtype = {'AREA_X': int, 'ID_CODE': int, 'MODE_FX': int, 
              'ST': int, 'SUB_REG': int, 'WAVE': int, 
              'YEAR': int, 'month': int, 'kod': str, 
              'SP_CODE': float, 'LNGTH': float, 'WGT': float, 
              'lngth_imp': int, 'wgt_imp': int, 'strat_id': str, 
              'psu_id': int, 'common': str, 'wgt_unadj': float, 
              'wp_size': float, 'l_in_bin': float, 'l_cm_bin': float, 
              'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
              'date_published': str}

In [78]:
@task()
def create_df(folder: str, bucket) -> list:
    """Get list of zip folders in GCS bucket"""
    lsblob = list(bucket.list_blobs(prefix=f"{folder}"))  # get list of blobs in zip folder of bucket
    lsblob = [l.name for l in lsblob]  # only return the filename from the blobs
    # lsblob = [l.name.split('/')[1] for l in lsblob]  # only return the filename from the blobs
    # strblob = ','.join(lsblob)
    # print(strblob)

    for blob in lsblob:
        with bucket.blob(blob).open('rb') as file:
            df = pd.read_csv(file)
            break
    return df

c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\tasks.py:275: UserWarning: A task named 'create_df' and defined at 'C:\Users\jlopez\AppData\Local\Temp\ipykernel_15792\2333926177.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [79]:
@flow(log_prints=False)
def write_bq(folder: str = "catch", bucket_name: str = "de_project_bucket") -> None:
    """Process blob in gcs bucket"""
    storage_client = storage.Client.from_service_account_json("../creds.json")
    # bucket_name = "de_project_bucket"  # parameterize this
    bucket = storage_client.get_bucket(bucket_name)

    df = create_df(folder, bucket)
    print(df.head())

c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\flows.py:214: UserWarning: A flow named 'write-bq' and defined at 'C:\Users\jlopez\AppData\Local\Temp\ipykernel_15792\1671885404.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [80]:
if __name__ == '__main__':
    folder = "size"
    bucket_name = "de_project_bucket"
    write_bq(folder, bucket_name)  



23:32:14.669 | INFO    | prefect.engine - Created flow run 'accomplished-ant' for flow 'write-bq'

23:32:16.096 | INFO    | Flow run 'accomplished-ant' - Created task run 'create_df-0' for task 'create_df'

23:32:16.106 | INFO    | Flow run 'accomplished-ant' - Executing 'create_df-0' immediately...

23:32:16.993 | INFO    | Task run 'create_df-0' - Finished in state Completed()

Empty DataFrame
Columns: [wave, year, st, mode_fx, sub_reg, sp_code, common, VAR_ID, ARX_METHOD, ALT_FLAG, STRAT_ID, date_published]
Index: []


23:32:17.189 | INFO    | Flow run 'accomplished-ant' - Finished in state Completed('All states completed.')